In [30]:
#import fiona
import pandas as pd
import geopandas
import os
import cenpy
acs = cenpy.products.ACS(2017)

D:\anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
os.chdir('C:/Users/90437/Documents/GitHub/SoCalEVCSNetwork')
os.getcwd()

'C:\\Users\\90437\\Documents\\GitHub\\SoCalEVCSNetwork'

# Data Importing and Cleaning

## SoCal Zipcodes and Geometry

In [3]:
socal_zipcodes = pd.read_csv (r'data/socal_zipcodes/socal_zip.csv')

In [4]:
socal_zipcodes = socal_zipcodes[['ZCTA5CE10','GEOID10','INTPTLAT10','INTPTLON10','geometry']]

In [5]:
socal_zipcodes = socal_zipcodes.rename(columns={"ZCTA5CE10": "Zip Code", "GEOID10": "GEOID", "INTPTLAT10": "Latitude",
                              "INTPTLON10": "Longitude", "geometry": "Geometry"})

In [6]:
socal_zipcodes.head()

,Zip Code,GEOID,Latitude,Longitude,Geometry
0,91504,691504,34.204568,-118.326365,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,91606,691606,34.185767,-118.388324,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,90640,690640,34.015444,-118.111012,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,91770,691770,34.065493,-118.084529,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,90290,690290,34.107728,-118.615981,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


In [27]:
socal_zipcodes['Zip Code'] = socal_zipcodes['Zip Code'].apply(str)

In [86]:
vars_to_download = {
    "B02001_002E": "total_pop_white",     # Total white population
    "B02001_003E": "total_pop_black",     #Total black population
    "B01003_001E": "total_pop",           # Total population
    "B09019_001E": "hh_total",            # Total households
    "B15003_002E": "total_bachelor",      # Total w/ Bachelor degree
    "B01002_001E": "median_age",          # Median age
    "B19013_001E": "median_hh_income",     # Median household income 
    "B19058_001E": "SNAP_hh",              # Households receiving Food Stamps/SNAP 
    "B08015_001E": "access_to_vehicle",   # Workers over age 16 that drove alone to work by car, van, truck
    "B99011_001E": "allocation of sex",   #no all 001,002,003
    "B99084_001E": "allocation of travel time to work",  #no all 001,002,003
    "B01001_001E": "sex by age"    #no all 001,002,003
    
}
vars_to_download_l = list(vars_to_download.keys())

In [87]:
db = acs.from_county("Orange, CA",
                  level="tract",
                  variables=vars_to_download_l
                 )

D:\anaconda\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [88]:
filler = lambda col: col.fillna(col.mean())
db.loc[:, vars_to_download] = db.loc[:, vars_to_download]\
                                .apply(filler)

In [89]:
db = db.rename(columns=vars_to_download)

In [90]:
db.head()

,GEOID,geometry,sex by age,median_age,total_pop,total_pop_white,total_pop_black,access_to_vehicle,hh_total,total_bachelor,median_hh_income,SNAP_hh,allocation of sex,allocation of travel time to work,NAME,state,county,tract
0,06059099504,"POLYGON ((-13147485.090 3995411.930, -13147414...",2661.0,49.1,2661.0,2282.0,0.0,890.0,2661.0,8.0,121295.0,945.0,2661.0,1129.0,"Census Tract 995.04, Orange County, California",06,059,099504
1,06059110108,"POLYGON ((-13145077.470 4002436.040, -13144880...",2895.0,46.9,2895.0,1897.0,95.0,1210.0,2895.0,13.0,105288.0,1056.0,2895.0,1408.0,"Census Tract 1101.08, Orange County, California",06,059,110108
2,06059110111,"POLYGON ((-13142748.890 4006204.470, -13142745...",6148.0,38.8,6148.0,3233.0,357.0,2570.0,6148.0,72.0,83289.0,1918.0,6148.0,3091.0,"Census Tract 1101.11, Orange County, California",06,059,110111
3,06059110203,"POLYGON ((-13138827.990 4004260.010, -13138780...",5049.0,38.3,5049.0,3443.0,0.0,2215.0,5049.0,55.0,79483.0,1661.0,5049.0,2469.0,"Census Tract 1102.03, Orange County, California",06,059,110203
4,06059087802,"POLYGON ((-13135930.340 4004301.010, -13135871...",7500.0,38.5,7500.0,4199.0,23.0,2770.0,7500.0,289.0,56674.0,2132.0,7500.0,3303.0,"Census Tract 878.02, Orange County, California",06,059,087802


In [34]:
acs.tables
# detail info in census

,description,columns
table_name,,
B00001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,[B00001_001E]
B00002,UNWEIGHTED SAMPLE HOUSING UNITS,[B00002_001E]
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B01003,TOTAL POPULATION,[B01003_001E]
...,...,...
C27014,PUBLIC HEALTH INSURANCE BY WORK EXPERIENCE,"[C27014_001E, C27014_002E, C27014_003E, C27014..."
C27016,HEALTH INSURANCE COVERAGE STATUS BY RATIO OF I...,"[C27016_001E, C27016_002E, C27016_003E, C27016..."
C27017,PRIVATE HEALTH INSURANCE BY RATIO OF INCOME TO...,"[C27017_001E, C27017_002E, C27017_003E, C27017..."
